In [ ]:
#This defines a class named Value that stores a scalar value and its gradient. It's used for automatic differentiation.

class Value:
    """ stores a single scalar value and its gradient """


## Initialization

In [ ]:
def __init__(self, data, _children=(), _op=''):
    self.data = data
    self.grad = 0
    # internal variables used for autograd graph construction
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op # the op that produced this node, for graphviz / debugging / etc


self.data: Stores the scalar value.

self.grad: Stores the gradient of the scalar value (initialized to 0).

self._backward: A function that will be used to compute the gradient during the backward pass.

self._prev: A set of parent nodes (used to keep track of the computational graph).

self._op: Stores the operation that created this node, useful for debugging.

## Addition

In [ ]:
# __add__: Defines addition for Value objects.
# Converts "other" to a "Value" if it is not already.
# Creates a new "Value" that represents the result of the addition.
# Defines a "_backward" function to propagate the gradients.
# Assigns the "_backward" function to the output Value.
def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
        self.grad += out.grad
        other.grad += out.grad
    out._backward = _backward

    return out


# Multiplication

In [ ]:
# __mul__: Defines multiplication for Value objects.
# Similar to addition, but also includes the chain rule for multiplication in _backward


def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
        self.grad += other.data * out.grad
        other.grad += self.data * out.grad
    out._backward = _backward

    return out


 ## Power

In [ ]:
#__pow__: Defines the power operation for Value objects.
# Only supports integer and float exponents.
#Includes the chain rule for the power operation in _backward.

def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += (other * self.data**(other-1)) * out.grad
    out._backward = _backward

    return out


## RELU (rectified Linear Unit)

In [ ]:
# relu: Implements the ReLU activation function.
# Sets the gradient to zero if the input is less than zero, otherwise passes the gradient through.
def relu(self):
    out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

    def _backward():
        self.grad += (out.data > 0) * out.grad
    out._backward = _backward

    return out



## Backward Pass

"backward": Computes the gradients for all nodes in the computational graph.

Uses a topological sort to ensure that gradients are computed in the correct order.

Initializes the gradient of the starting node to 1 (since the derivative of itself is 1).

Applies the "_backward" function for each node in the topologically sorted order.

In [ ]:
def backward(self):
    # topological order all of the children in the graph
    topo = []
    visited = set()

    def build_topo(v):
        if v not in visited:
            visited.add(v)
            for child in v._prev:
                build_topo(child)
            topo.append(v)
    build_topo(self)

    # go one variable at a time and apply the chain rule to get its gradient
    self.grad = 1
    for v in reversed(topo):
        v._backward()


## Other methods

In [ ]:
#These methods define additional arithmetic operations (negation, addition, subtraction, multiplication, division) and their corresponding reverse operations.
# __repr__: Provides a string representation of the Value object for easy debugging.

def __neg__(self): # -self
    return self * -1

def __radd__(self, other): # other + self
    return self + other

def __sub__(self, other): # self - other
    return self + (-other)

def __rsub__(self, other): # other - self
    return other + (-self)

def __rmul__(self, other): # other * self
    return self * other

def __truediv__(self, other): # self / other
    return self * other**-1

def __rtruediv__(self, other): # other / self
    return other * self**-1

def __repr__(self):
    return f"Value(data={self.data}, grad={self.grad})"


## Summary

This class Value is designed for a simple automatic differentiation system, allowing youperform operations on scalar values and automatically compute their gradients. This is useful for implementing machine learning algorithms that require gradient-based optimization methods

## Implementing a Graph Neural Network (GNN) using the Value class for automatic differentiation.

 The processinvolves several steps.
 
  GNNs are designed to work with graph-structured data, where nodes represent entities and edges represent relationships between them. 
  
  The Value class need to be adapted to handle the computations in a GNN. 
  
  Here’s a high-level overview and an example of how to integrate the Value class into a simple GNN framework:

High-Level Steps
1. Define the Graph Structure: Represent the graph using nodes and edges. It can be with an adjacency matrix or a adjacency list. It can be also provided.

2. Node Features and Initialization: Initialize the node features as Value objects.

3. Message Passing: Implement message passing where node features are updated based on the features of neighboring nodes.

4. Aggregation: Aggregate messages from neighbors.

5. Update: Update the node features using the aggregated messages.

5. Loss Calculation and Backpropagation: Compute the loss and use the backward method to compute gradients.

This example outlines how to integrate the Value class into a basic GNN framework. The GNN layer performs message passing, aggregation, and updates the node features. The loss function is defined, and backpropagation is used to compute gradients. This implementation can be expanded and refined to include more complex GNN architectures and tasks, such as node classification or link prediction.

## 1. Define the Graph Structure
A simple graph representation with nodes and edges

In [ ]:
class Node:
    def __init__(self, features):
        self.features = Value(features)

class Edge:
    def __init__(self, src, dst):
        self.src = src
        self.dst = dst


## 2. Node Features and Initialization
Initialize the nodes with some feature values.

In [ ]:
nodes = [Node(1.0), Node(2.0), Node(3.0)]
edges = [Edge(nodes[0], nodes[1]), Edge(nodes[1], nodes[2])]


## 3. Message Passing
Define a simple message-passing function.

In [ ]:
def message(src_node, dst_node):
    return src_node.features * 0.5  # Simple example where message is half the source node's features


## 4. Aggregation
Aggregate messages from all neighbors. For simplicity, we'll sum the messages.

In [ ]:
def aggregate(messages):
    result = Value(0.0)
    for msg in messages:
        result = result + msg
    return result


## 5. Update
Update node features based on aggregated messages

In [ ]:
def update(node, aggregated_message):
    node.features = node.features + aggregated_message  # Simple update rule


## 6 GNN Layer Implementation
Combine the above steps into a GNN layer.

In [ ]:
def gnn_layer(nodes, edges):
    # Step 1: Message Passing
    messages = {node: [] for node in nodes}
    for edge in edges:
        msg = message(edge.src, edge.dst)
        messages[edge.dst].append(msg)
    
    # Step 2: Aggregation and Update
    for node in nodes:
        aggregated_message = aggregate(messages[node])
        update(node, aggregated_message)


## 7. Loss Calculation and Backpropagation
Define a simple loss and perform backpropagation.

In [ ]:
# Define a simple loss function (e.g., sum of node features)
def loss(nodes):
    total_loss = Value(0.0)
    for node in nodes:
        total_loss = total_loss + node.features
    return total_loss

# Run the GNN layer
gnn_layer(nodes, edges)

# Compute the loss
total_loss = loss(nodes)

# Perform backpropagation
total_loss.backward()

# Print gradients
for node in nodes:
    print(node.features.grad)
